This is a revamped beta version of NLG module for CAiFE without using Transformers libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir("/content/drive/MyDrive/")

In [ ]:
import json
import re
import random

In [ ]:
# Create a dafault response in the case that user does not enter anything
def random_default_response():
    random_list = [
        "Please give me more details on what you need.",
        "Oh! It appears you wrote something I don't understand yet",
        "Do you mind trying to rephrase that?",
        "I'm terribly sorry, I didn't quite catch that.",
        "I can't answer that yet, could you please be more specific?"
    ]

    list_count = len(random_list)
    random_item = random.randrange(list_count)

    return random_list[random_item]


In [ ]:
# Load JSON data
def load_json(file):
    with open(file) as bot_responses:
        print(f"Loaded '{file}' successfully!")
        return json.load(bot_responses)

In [ ]:
# Store JSON data "bert2_dialogue_016" as provided by Kumar
# intents_data = load_json("bert2_dialogue_016.json")
intents_data = load_json("bot.json")

Loaded 'bot.json' successfully!


In [ ]:
intents_data[0]

{'dialogue_idx': 'MUL1547.json',
 'domains': ['restaurant'],
 'dialogue': [{'system_transcript': '',
   'turn_idx': 0,
   'belief_state': [{'slots': [['restaurant-area', 'west']],
     'act': 'find_restaurant'},
    {'slots': [['restaurant-food', 'indian']], 'act': 'find_restaurant'}],
   'turn_label': [['restaurant-area', 'west'], ['restaurant-food', 'indian']],
   'transcript': "I'm looking for an Indian restaurant in the west end of Cambridge.",
   'system_acts': [],
   'domain': 'restaurant'},
  {'system_transcript': 'There are 6 results for Indian restaurants in the west, what price range are you looking for?',
   'turn_idx': 1,
   'belief_state': [{'slots': [['restaurant-area', 'west']],
     'act': 'find_restaurant'},
    {'slots': [['restaurant-food', 'indian']], 'act': 'find_restaurant'}],
   'turn_label': [['restaurant-area', 'west'], ['restaurant-food', 'indian']],
   'transcript': 'I do not have a preference, so surprise me!',
   'system_acts': ['restaurant-name'],
   'doma

In [ ]:
domain = intents_data[1]['dialogue'][0]['domain']
print(domain)

restaurant


In [ ]:
intents_data[1]['dialogue'][0]['belief_state'][0]['slots'][0][0].split("-")

['restaurant', 'food']

In [ ]:
act = intents_data[1]['dialogue'][0]['belief_state'][0]['act'].split("_")
print(act)

['find', 'restaurant']


In [ ]:
act = intents_data[1]['dialogue'][0]['belief_state'][0]['act'].split("_")
print(act)

In [ ]:
user_dialogue = intents_data[0]['dialogue'][0]['transcript']
print(user_dialogue)

I'm looking for an Indian restaurant in the west end of Cambridge.


In [ ]:
slot_value = intents_data[0]['dialogue'][0]['belief_state'][0]['slots'][0][0].split("-")
print(slot_value)

['restaurant', 'area']


In [ ]:
intents_data[0]['dialogue'][0]['belief_state'][0]['slots'][0][1].split("-")


['west']

In [ ]:
def generate_NLG_response(input_string):
    user_message = re.split(r'\s+|[,;?!.-]\s*', input_string.lower())
    score_list = []

    # Check all the intents
    for intents in intents_data:
        response_score = 0
        required_score = 0
        required_words = intents["slots"]

        # Check if there are any required slot-filling words
        if required_words:
            for word in user_message:
                if word in required_words:
                    required_score += 1

        # Amount of required words should match the required score -> slot is fully filled
        if required_score == len(required_words):
            # print(required_score == len(required_words))
            # Check each word the user has typed
            for word in user_message:
                # If the word is in the response, add to the score
                if word in intents["transcript"]:
                    response_score += 1

        # Add score to list
        score_list.append(response_score)
        # Debugging: Find the best phrase
        # print(response_score, response["user_input"])

    # Find the best response and return it if they're not all 0
    best_response = max(score_list)
    response_index = score_list.index(best_response)

    # Check if input is empty
    if input_string == "":
        return "Please tell us how we could be of help."

    # If there is no good response, return a random one.
    if best_response != 0:
        return intents_data[response_index]["system_transcript"]

    return random_default_response()


In [ ]:
while True:
    user_input = input("USR: ")
    print("BOT:", generate_NLG_response(user_input))

USR: hi
BOT: Hey there!
USR: I need vaccination
BOT: Did you have any vacccination before?
USR: yes I did
BOT: Oh! It appears you wrote something I don't understand yet
USR: yes I did have vaccination
BOT: Sure, when do you want to have it?
USR: how about Thursday?
BOT: Oh! It appears you wrote something I don't understand yet
USR: Can you help to book it Wednesday?
BOT: I can't answer that yet, please try to rephrase.
USR: Can you help to book it next Wednesday?
BOT: Do you mind trying to rephrase that?
USR: Thursday
BOT: I can't answer that yet, please try to rephrase.
USR: 3pm
BOT: I'm terribly sorry, I didn't quite catch that.
USR: book 3pm
BOT: Please give me more details on what you need.
USR: book time at 3pm
BOT: Sure, your booking is succesful.
USR: thank you
BOT: I'm terribly sorry, I didn't quite catch that.
USR: Thank you
BOT: I'm terribly sorry, I didn't quite catch that.
USR: Thank you, good bye
BOT: See you later!


KeyboardInterrupt: ignored